In [10]:
path = os.path.join(os.getcwd(), 'new_data')

In [17]:
gesns = np.empty(0, dtype=str)B

In [ ]:
for table in os.listdir(path):
        table_path = os.path.join(path, table)
        df = pd.read_excel(table_path, dtype={"Шифр": str})
        df.dropna(subset=["Шифр"], inplace=True)
        gesns = np.union1d(gesns, df.Шифр.unique())
        # gesns = df.Шифр.unique()
gesns.size

In [30]:
historical_df = pd.read_excel("data/2022-02-07 МОЭК_ЕКС график по смете.xlsx")
historical_df.dropna(subset=["ADCM_шифрГЭСН"], inplace=True)
historical_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 3 to 339
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Идентификатор операции  292 non-null    object        
 1   ИСР                     292 non-null    object        
 2   Наименование            292 non-null    object        
 3   Длительность - план     292 non-null    int64         
 4   Начало                  292 non-null    datetime64[ns]
 5   Окончание               292 non-null    datetime64[ns]
 6   ADCM_Объем план         292 non-null    float64       
 7   ADCM_Стоимость_План     292 non-null    object        
 8   ADCM_шифрГЭСН           292 non-null    object        
 9   ADCM_МаркаРД            292 non-null    object        
 10  Последователи           229 non-null    object        
 11  Предшественники         272 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object

In [31]:
hist_gesns = historical_df.ADCM_шифрГЭСН.unique()
hist_gesns.size

133

In [32]:
np.intersect1d(hist_gesns, gesns)

array(['15.2-1-7', '2.1-3-38', '3.1-2-10', '3.1-29-1', '3.1-30-1',
       '3.1-49-1', '3.1-53-1', '3.1-6-10', '3.11-10-1', '3.13-11-5',
       '3.13-11-8', '3.13-9-2', '3.16-9-1', '3.16-9-3', '3.18-20-2',
       '3.18-20-4', '3.22-43-1', '3.22-70-1', '3.22-70-3', '3.22-71-1',
       '3.22-72-2', '3.23-1-5', '3.23-13-1', '3.23-13-3', '3.23-24-1',
       '3.23-28-1', '3.24-2-1', '3.24-2-3', '3.24-38-1', '3.24-40-1',
       '3.24-41-1', '3.24-42-1', '3.24-43-1', '3.26-11-1', '3.27-26-1',
       '3.27-30-1', '3.27-30-2', '3.27-42-1', '3.27-43-1', '3.27-54-1',
       '3.29-1587-2', '3.29-1588-1', '3.29-1590-2', '3.29-1591-1',
       '3.29-1596-1', '3.29-1596-2', '3.4-10-2', '3.4-11-2', '3.4-19-36',
       '3.4-6-10', '3.4-8-53', '3.47-26-3', '3.47-26-4', '3.47-26-5',
       '3.47-26-6', '3.47-3-15', '3.47-3-20', '3.47-3-5', '3.47-34-6',
       '3.47-5-15', '3.47-5-20', '3.47-5-5', '3.47-7-5', '3.47-7-7',
       '3.47-7-8', '3.6-1-1', '3.6-1-15', '3.6-8-1', '3.7-21-2',
       '3.7-45-1', '3.

In [1]:
import os

import pandas as pd
import numpy as np
from neo4j import Transaction, Session, GraphDatabase

In [2]:
df1 = pd.read_excel("new_data/Сметы Берзарина.xls")
df1.head(1)

,СППэлемент,Проект,[ GUID локальной сметы],Смета,Код,Наименование,НаименованиеПолное,Шифр,Пункт,Тип работ,Стоимость всего,Единица измерения,Давальческие материалы
0,M-20-1A0-09-10-000032,Реконструкция магистральных т/сетей,2cad624d-c19c-11ed-a2ca-005056936a79,Локальный сметный расчет №01-01-01 от 01.09.20...,00-00000011,Валка кустарников - 8 шт.,Валка кустарников - 8 шт.,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df1.rename(
    columns={"Проект": "wbs1", "Смета": "wbs2", "Шифр": "wbs3_id", "Наименование": "name"},
    inplace=True
)
df1 = df1[["wbs1", "wbs2", "wbs3_id", "name"]]

In [4]:
df1.nunique()

wbs1         1
wbs2        17
wbs3_id    408
name       759
dtype: int64

In [5]:
df1.shape

(1440, 4)

In [6]:
df1.dropna(subset=["wbs3_id"], inplace=True)
df1.drop_duplicates(inplace=True)
df1.shape

(720, 4)

In [7]:
df1.nunique()

wbs1         1
wbs2        17
wbs3_id    408
name       645
dtype: int64

In [8]:
os.getcwd()

'C:\\Users\\naumo\\PycharmProjects\\graph_roder'

In [ ]:
from

In [15]:
# Q_NODES_OBTAIN = '''
# MATCH (n)
# WHERE n.type = 'start'
# RETURN n.name AS name, n.DIN AS din, n.type AS type
# '''
# Q_RELS_OBTAIN = '''
# MATCH (n)-[r:FOLLOWS]->(m)
# RETURN n.DIN AS n_din, n.type AS n_type,
# m.DIN AS m_din, m.type AS m_type,
# properties(r).weight AS weight
# '''
HIST_URI = "neo4j+s://99c1a702.databases.neo4j.io:7687"
F_URI = "neo4j://127.0.0.1:7688"
USER = "neo4j"
PSWD = "231099"
hist_driver = GraphDatabase.driver(HIST_URI, auth=(USER, PSWD))
with hist_driver.session() as session:
    nodes = pd.DataFrame(session.run(Q_NODES_OBTAIN).data())
    rels = pd.DataFrame(session.run(Q_RELS_OBTAIN).data())
hist_driver.close()

In [16]:
nodes.to_csv(os.path.join(os.getcwd(), "nodes.csv"), index=False)
nodes.shape

(180, 3)

In [17]:
rels.to_csv(os.path.join(os.getcwd(), "rels.csv"), index=False)
rels.shape

(252, 5)